In [1]:
!pip install torchsummary
!pip install torchgeometry

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.7/42.7 kB 2.1 MB/s eta 0:00:00


In [2]:
from torchsummary import summary
from torchgeometry.losses import one_hot
import os
import numpy as np
import pandas as pd
from PIL import Image
import cv2
import time
import imageio
import matplotlib.pyplot as plt
import time
from torch.utils.data import  ConcatDataset, Dataset, DataLoader, random_split
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch import Tensor
from torchvision.transforms import *
from collections import OrderedDict
import albumentations as A
from albumentations.pytorch import ToTensorV2
import wandb


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-4a6398af-93fd-7cc5-d7f1-075aa03a0bd1)
GPU 1: Tesla T4 (UUID: GPU-29ba221e-0c5e-1e73-b21b-e85079ee9ca6)


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

# Parameters

In [5]:
# Number of class in the data set (3: neoplastic, non neoplastic, background)
num_classes = 3

# Number of epoch
epochs = 20

# Hyperparameters for training 
learning_rate = 2e-04
batch_size = 4
display_step = 50

# Model path
checkpoint_path = '/kaggle/working/unet_model.pth'
pretrained_path = "/kaggle/input/unet-checkpoint/unet_model.pth"
# Initialize lists to keep track of loss and accuracy
loss_epoch_array = []
train_accuracy = []
test_accuracy = []
valid_accuracy = []

# Dataloader

In [6]:
transform = Compose([Resize((256, 256), interpolation=InterpolationMode.BILINEAR),
                     PILToTensor()])

transform1  = Compose([ Resize((256, 256), interpolation=InterpolationMode.BILINEAR),
                        RandomVerticalFlip(p=0.9),
                       RandomAffine((20,50)),
                       GaussianBlur(kernel_size=3),
                        PILToTensor()])
                   
transform2 = Compose([ Resize((256, 256), interpolation=InterpolationMode.BILINEAR),
                       RandomHorizontalFlip(p=0.9),
                       RandomRotation(degrees=20),
                      ColorJitter(brightness=0.5, contrast=1, saturation=0.1, hue=0.5),
                        PILToTensor()])

In [7]:
class UNetDataClass(Dataset):
    def __init__(self, images_path, masks_path, transform=None):
        super(UNetDataClass, self).__init__()
        
        images_list = os.listdir(images_path)
        masks_list = os.listdir(masks_path)
        
        images_list = [images_path + image_name for image_name in images_list]
        masks_list = [masks_path + mask_name for mask_name in masks_list]
        
        self.images_list = images_list
        self.masks_list = masks_list
        self.transform = transform
        
    def __getitem__(self, index):
        img_path = self.images_list[index]
        mask_path = self.masks_list[index]
        
        # Open image and mask
        data = Image.open(img_path)
        label = Image.open(mask_path)
            
        if (self.transform):
            data = self.transform(data) / 255
            label = self.transform(label) / 255

        
        label = torch.where(label>0.65, 1.0, 0.0)
        
        label[2, :, :] = 0.0001
        label = torch.argmax(label, 0).type(torch.int64)
        
        return data, label
    
    def __len__(self):
        return len(self.images_list)



In [8]:
images_path = "/kaggle/input/bkai-igh-neopolyp/train/train/"
masks_path =  "/kaggle/input/bkai-igh-neopolyp/train_gt/train_gt/"

In [9]:
original_dataset = UNetDataClass(images_path, masks_path, transform=transform)
dataset_transform1 = UNetDataClass(images_path, masks_path, transform=transform1)
dataset_transform2 = UNetDataClass(images_path, masks_path, transform=transform2)

dataset1 = ConcatDataset([original_dataset, dataset_transform1])
unet_dataset = ConcatDataset([dataset1, dataset_transform2])

In [10]:
train_size = 0.9
valid_size = 0.1

In [11]:
train_set, valid_set = random_split(unet_dataset, 
                                    [int(train_size * len(unet_dataset)) , 
                                     int(valid_size * len(unet_dataset))])

In [12]:
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True)

# Loss function

In [13]:
class CEDiceLoss(nn.Module):
    def __init__(self, weights) -> None:
        super(CEDiceLoss, self).__init__()
        self.eps: float = 1e-6
        self.weights: torch.Tensor = weights

    def forward(
            self,
            input: torch.Tensor,
            target: torch.Tensor) -> torch.Tensor:
        if not torch.is_tensor(input):
            raise TypeError("Input type is not a torch.Tensor. Got {}"
                            .format(type(input)))
        if not len(input.shape) == 4:
            raise ValueError("Invalid input shape, we expect BxNxHxW. Got: {}"
                             .format(input.shape))
        if not input.shape[-2:] == target.shape[-2:]:
            raise ValueError("input and target shapes must be the same. Got: {}"
                             .format(input.shape, input.shape))
        if not input.device == target.device:
            raise ValueError(
                "input and target must be in the same device. Got: {}" .format(
                    input.device, target.device))
        if not self.weights.shape[1] == input.shape[1]:
            raise ValueError("The number of weights must equal the number of classes")
        if not torch.sum(self.weights).item() == 1:
            raise ValueError("The sum of all weights must equal 1")
            
        # cross entropy loss
        celoss = nn.CrossEntropyLoss(self.weights)(input, target)
        
        # compute softmax over the classes axis
        input_soft = F.softmax(input, dim=1)

        # create the labels one hot tensor
        target_one_hot = one_hot(target, num_classes=input.shape[1],
                                 device=input.device, dtype=input.dtype)

        # compute the actual dice score
        dims = (2, 3)
        intersection = torch.sum(input_soft * target_one_hot, dims)
        cardinality = torch.sum(input_soft + target_one_hot, dims)

        dice_score = 2. * intersection / (cardinality + self.eps)
        
        dice_score = torch.sum(dice_score * self.weights, dim=1)
        
        return torch.mean(1. - dice_score) + celoss

# Training

**Initialize weights**

In [14]:
def weights_init(model):
    if isinstance(model, nn.Linear):
        # Xavier Distribution
        torch.nn.init.xavier_uniform_(model.weight)

In [15]:
def save_model(model, optimizer, path):
    checkpoint = {
        "model": model.state_dict(),
        "optimizer": optimizer.state_dict(),
    }
    torch.save(checkpoint, path)

def load_model(model, optimizer, path):
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint["model"])
    optimizer.load_state_dict(checkpoint['optimizer'])
    return model, optimizer

**Train model**

In [16]:
# Train function for each epoch
def train(train_dataloader, valid_dataloader,learing_rate_scheduler, epoch, display_step):
    print(f"Start epoch #{epoch+1}, learning rate for this epoch: {learing_rate_scheduler.get_last_lr()}")
    start_time = time.time()
    train_loss_epoch = 0
    test_loss_epoch = 0
    last_loss = 999999999
    model.train()
    for i, (data,targets) in enumerate(train_dataloader):
        
        # Load data into GPU
        data, targets = data.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(data)

        # Backpropagation, compute gradients
        loss = loss_function(outputs, targets.long())
        loss.backward()

        # Apply gradients
        optimizer.step()
        
        # Save loss
        train_loss_epoch += loss.item()
        if (i+1) % display_step == 0:
#             accuracy = float(test(test_loader))
            print('Train Epoch: {} [{}/{} ({}%)]\tLoss: {:.4f}'.format(
                epoch + 1, (i+1) * len(data), len(train_dataloader.dataset), 100 * (i+1) * len(data) / len(train_dataloader.dataset), 
                loss.item()))
                  
    print(f"Done epoch #{epoch+1}, time for this epoch: {time.time()-start_time}s")
    train_loss_epoch/= (i + 1)
    
    # Evaluate the validation set
    model.eval()
    with torch.no_grad():
        for data, target in valid_dataloader:
            data, target = data.to(device), target.to(device)
            test_output = model(data)
            test_loss = loss_function(test_output, target)
            test_loss_epoch += test_loss.item()
            
    test_loss_epoch/= (i+1)
    
    return train_loss_epoch , test_loss_epoch

In [17]:
!pip install -q segmentation-models-pytorch
import segmentation_models_pytorch as smp

model = smp.UnetPlusPlus(
    encoder_name='resnet50', 
    encoder_weights="imagenet",     
    in_channels=3,                  
    classes=3     
)

model.apply(weights_init)
# # model = nn.DataParallel(model)
# checkpoint = torch.load(pretrained_path)

# new_state_dict = OrderedDict()
# for k, v in checkpoint['model'].items():
#     name = k[7:] # remove `module.`
#     new_state_dict[name] = v
# # load params
# model.load_state_dict(new_state_dict)
model = nn.DataParallel(model)
model.to(device)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 227MB/s] 


DataParallel(
  (module): UnetPlusPlus(
    (encoder): ResNetEncoder(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu

In [18]:
weights = torch.Tensor([[0.4, 0.55, 0.05]]).cuda()
loss_function = CEDiceLoss(weights)

# Define the optimizer (Adam optimizer)
optimizer = optim.Adam(params=model.parameters(), lr=learning_rate)
# optimizer.load_state_dict(checkpoint['optimizer'])

# Learning rate scheduler
learing_rate_scheduler = lr_scheduler.StepLR(optimizer, step_size=4, gamma=0.6)

In [19]:
save_model(model, optimizer, checkpoint_path)

In [20]:
wandb.login(
    key = "f49e2b5c71203e093d9349c33d5b052b8e9267e8",
)
wandb.init(
    project = "DLassignment"
)

# Training loop
train_loss_array = []
test_loss_array = []
last_loss = 9999999999999
for epoch in range(epochs):
    train_loss_epoch = 0
    test_loss_epoch = 0
    (train_loss_epoch, test_loss_epoch) = train(train_loader, 
                                              valid_loader, 
                                              learing_rate_scheduler, epoch, display_step)
    
    if test_loss_epoch < last_loss:
        save_model(model, optimizer, checkpoint_path)
        last_loss = test_loss_epoch
        
    learing_rate_scheduler.step()
    train_loss_array.append(train_loss_epoch)
    test_loss_array.append(test_loss_epoch)
    wandb.log({"Train loss": train_loss_epoch, "Valid loss": test_loss_epoch})


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: nguyentrungtru. Use `wandb login --relogin` to force relogin


Start epoch #1, learning rate for this epoch: [0.0002]
Train Epoch: 1 [200/2700 (7.407407407407407%)]	Loss: 1.6331
Train Epoch: 1 [400/2700 (14.814814814814815%)]	Loss: 1.4536
Train Epoch: 1 [600/2700 (22.22222222222222%)]	Loss: 1.4867
Train Epoch: 1 [800/2700 (29.62962962962963%)]	Loss: 1.4162
Train Epoch: 1 [1000/2700 (37.03703703703704%)]	Loss: 1.4758
Train Epoch: 1 [1200/2700 (44.44444444444444%)]	Loss: 1.2000
Train Epoch: 1 [1400/2700 (51.851851851851855%)]	Loss: 1.0515
Train Epoch: 1 [1600/2700 (59.25925925925926%)]	Loss: 1.3194
Train Epoch: 1 [1800/2700 (66.66666666666667%)]	Loss: 0.9755
Train Epoch: 1 [2000/2700 (74.07407407407408%)]	Loss: 1.0734
Train Epoch: 1 [2200/2700 (81.48148148148148%)]	Loss: 1.0116
Train Epoch: 1 [2400/2700 (88.88888888888889%)]	Loss: 1.2611
Train Epoch: 1 [2600/2700 (96.29629629629629%)]	Loss: 1.1115
Done epoch #1, time for this epoch: 271.34675431251526s
Start epoch #2, learning rate for this epoch: [0.0002]
Train Epoch: 2 [200/2700 (7.407407407407407

In [21]:


def rle_to_string(runs):
    return ' '.join(str(x) for x in runs)

def rle_encode_one_mask(mask):
    pixels = mask.flatten()
    pixels[pixels > 0] = 255
    use_padding = False
    if pixels[0] or pixels[-1]:
        use_padding = True
        pixel_padded = np.zeros([len(pixels) + 2], dtype=pixels.dtype)
        pixel_padded[1:-1] = pixels
        pixels = pixel_padded
    
    rle = np.where(pixels[1:] != pixels[:-1])[0] + 2
    if use_padding:
        rle = rle - 1
    rle[1::2] = rle[1::2] - rle[:-1:2]
    return rle_to_string(rle)

def rle2mask(mask_rle, shape=(3,3)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (width,height) of array to return 
    Returns numpy array, 1 - mask, 0 - background
    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T

def mask2string(dir):
    ## mask --> string
    strings = []
    ids = []
    ws, hs = [[] for i in range(2)]
    for image_id in os.listdir(dir):
        id = image_id.split('.')[0]
        path = os.path.join(dir, image_id)
        print(path)
        img = cv2.imread(path)[:,:,::-1]
        h, w = img.shape[0], img.shape[1]
        for channel in range(2):
            ws.append(w)
            hs.append(h)
            ids.append(f'{id}_{channel}')
            string = rle_encode_one_mask(img[:,:,channel])
            strings.append(string)
    r = {
        'ids': ids,
        'strings': strings,
    }
    return r


In [22]:


class UNetTestDataClass(Dataset):
    def __init__(self, images_path, transform):
        super(UNetTestDataClass, self).__init__()
        
        images_list = os.listdir(images_path)
        images_list = [images_path+i for i in images_list]
        
        self.images_list = images_list
        self.transform = transform
        
    def __getitem__(self, index):
        img_path = self.images_list[index]
        data = Image.open(img_path)
        h = data.size[1]
        w = data.size[0]
        data = self.transform(data) / 255        
        return data, img_path, h, w
    
    def __len__(self):
        return len(self.images_list)

In [23]:

from collections import OrderedDict
import segmentation_models_pytorch as smp
from collections import OrderedDict

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = smp.UnetPlusPlus(
    encoder_name='resnet50', 
    encoder_weights="imagenet",     
    in_channels=3,                  
    classes=3     
)

checkpoint = torch.load('unet_model.pth')

new_state_dict = OrderedDict()
for k, v in checkpoint['model'].items():
    name = k[7:] # remove `module.`
    new_state_dict[name] = v
# load params
model.load_state_dict(new_state_dict)



transform = Compose([Resize((256, 256), interpolation=InterpolationMode.BILINEAR),
                     PILToTensor()]) 


test_path = '/kaggle/input/bkai-igh-neopolyp/test/test/'
unet_test_dataset = UNetTestDataClass(test_path, transform)
test_dataloader = DataLoader(unet_test_dataset, batch_size=8, shuffle=True)


model.eval()
if not os.path.isdir("/kaggle/working/predicted_masks"):
    os.mkdir("/kaggle/working/predicted_masks")
for _, (img, path, H, W) in enumerate(test_dataloader):
    a = path
    b = img
    h = H
    w = W
    
    with torch.no_grad():
        predicted_mask = model(b)
    for i in range(len(a)):
        image_id = a[i].split('/')[-1].split('.')[0]
        filename = image_id + ".png"
        mask2img = Resize((h[i].item(), w[i].item()), interpolation=InterpolationMode.NEAREST)(ToPILImage()(F.one_hot(torch.argmax(predicted_mask[i], 0)).permute(2, 0, 1).float()))
        mask2img.save(os.path.join("/kaggle/working/predicted_masks/", filename))


MASK_DIR_PATH = '/kaggle/working/predicted_masks' 
dir = MASK_DIR_PATH
res = mask2string(dir)
df = pd.DataFrame(columns=['Id', 'Expected'])
df['Id'] = res['ids']
df['Expected'] = res['strings']
df.to_csv(r'output.csv', index=False)

/kaggle/working/predicted_masks/e56a6d9ba9d45c3dbc695325ded465ef.png
/kaggle/working/predicted_masks/d3694abb47953b0e4909384b57bb6a05.png
/kaggle/working/predicted_masks/395e56a6d9ba9d45c3dbc695325ded46.png
/kaggle/working/predicted_masks/05734fbeedd0f9da760db74a29abdb04.png
/kaggle/working/predicted_masks/d694539ef2424a9218697283baa3657e.png
/kaggle/working/predicted_masks/66e057db382b8564872a27301a654864.png
/kaggle/working/predicted_masks/7ad1cf2eb9d32a3dc907950289e976c7.png
/kaggle/working/predicted_masks/6f67b5df7cdf3f33c3ca4d5060a633a8.png
/kaggle/working/predicted_masks/be86f03d900fd197cd955fa095f97845.png
/kaggle/working/predicted_masks/e7998934d417cb2eb1ef57af2ed9fbb6.png
/kaggle/working/predicted_masks/a6d9ba9d45c3dbc695325ded465efde9.png
/kaggle/working/predicted_masks/98da48d679d7c7c8d3d96fb2b87fbbcf.png
/kaggle/working/predicted_masks/1b62f15ec83b97bb11e8e0c4416c1931.png
/kaggle/working/predicted_masks/80cae6daedd989517cb8041ed86e5822.png
/kaggle/working/predicted_masks/7f